In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from networkx.readwrite.gexf import read_gexf
import pandas as pd
from tqdm import tqdm
from collections import Counter
from sklearn.preprocessing import normalize

### Utility

In [3]:
def read_graph(path):
    '''
        read a networkx .gexf graph 
    '''
    g = read_gexf(path, node_type=type(1))
    adj_mat = nx.linalg.graphmatrix.adjacency_matrix(g)
    idmap = list(dict(g.nodes(data='false')).keys())    # mapping from the adj matrix index to node id 
    idmap_inv = dict(zip(idmap, [_ for _ in range(len(idmap))])) # mapping from node id to the adj matrix index
    return idmap, idmap_inv, adj_mat

def Sim(adj_mat, weighted=True, gamma=0.2):
    '''
        the similarity matrix, originally implemented by https://github.com/blmoistawinde/facetnet-python
    '''
    N = adj_mat.shape[0]
    sim = np.matrix(np.eye(N,N))
    if weighted:                           # as stated in Section 4.2
        #sim = np.where(adj_mat.toarray()>0, np.exp(-1.0/(gamma*adj_mat.toarray())), 1)
        sim = np.exp(-1.0/(gamma*adj_mat.toarray() + epsilon))
    else:
        # similarity measure for unweighted graph is not mentioned in the paper, below is my assumption, which seems to work.
        sim = adj_mat.toarray()/2.0                   # if linked, sim=0.5
    for i in range(N):
        sim[i,i] = 1.0
    sim /= np.sum(sim)
    sim = np.matrix(sim)
    return sim

def KL(A, B):
    '''
        KL-divergence 
    '''
    ret = 0.0
    for a in np.ravel(A):
        for b in np.ravel(B):
            ret += a*np.log(a/b) - a + b
    return ret

def soft_modularity(soft_comm,W):
    N = W.shape[0]
    ret = np.trace(soft_comm.T*W*soft_comm)
    one = np.matrix(np.ones((N,1)))
    ret -= np.array(one.T*W.T*soft_comm*soft_comm.T*W*one).squeeze()
    return ret

### FacetNet 1

Only the number of nodes would change (increase or decrease).

In [4]:
def param_update(X, A, Y, W, t, alpha, epoch):
    '''
        optimize the code from https://github.com/blmoistawinde/facetnet-python by derive 
            the iteration equations in form of matrix. greatly improve the efficiency.
    '''
    X_pre, A_pre, Y_pre = X,A,Y 
    N, M = Y.shape
    #epoch = max(int(epoch * (0.95**t) ), 3) 
    epsilon = 1e-40
    if t == 0:
        for e in tqdm(range(epoch)):
            W_apprx = np.true_divide(W , (X_pre * A_pre * X_pre.T+epsilon))
            W_apprx_X = W_apprx * X_pre
            X_new, A_new = X_pre, A_pre
            
            X_new = np.multiply(np.multiply(W_apprx_X, X_pre), np.sum(A_pre, axis=0).reshape(1, M))
            A_new = np.multiply(A_pre, np.diag(np.diag(X_pre.T * W_apprx * X_pre)))
                
            X_new, A_new = np.matrix(X_new / np.sum(X_new, axis=0).reshape(1, M)), np.matrix(A_new / np.sum(A_new))
            X_pre, A_pre = X_new, A_new
            
            
            # print epoch result
            Y_prime = X_new * A_new
            D = np.zeros((N,))
            for i in range(N):
                D[i] = np.sum(Y_prime[i, :])
            D = np.matrix(np.diag(D))
            soft_comm = D.I * X_new * A_new
            s_modu = soft_modularity(soft_comm, W)
            print("soft_modularity: %f" % s_modu)
            
    else:
        for e in tqdm(range(epoch)):
            try:
                W_apprx = np.true_divide(W , (X_pre * A_pre * X_pre.T + epsilon))
            except:
                print(np.where((X_pre * A_pre * X_pre.T) == 0 ))
            W_apprx_X = W_apprx * X_pre
            X_new, A_new = X_pre, A_pre
                        
            X_new = 2 * alpha * np.multiply(np.multiply(W_apprx_X, X_pre), np.sum(A_pre, axis=0).reshape(1, M)) + (1-alpha)* Y
            A_new = alpha * np.multiply(A_pre, np.diag(np.diag(X_pre.T * W_apprx * X_pre))) + (1-alpha) * np.diag(np.sum(Y, axis=1))
            X_new, A_new = np.matrix(X_new / np.sum(X_new, axis=0).reshape(1, M)), np.matrix(A_new / np.sum(A_new))
            X_pre, A_pre = X_new, A_new
            
            # print epoch result
            Y_prime = X_new * A_new
            D = np.zeros((N,))
            for i in range(N):
                D[i] = np.sum(Y_prime[i, :])
            D = np.matrix(np.diag(D))
            soft_comm = D.I * X_new * A_new
            s_modu = soft_modularity(soft_comm, W)
            print("soft_modularity: %f" % s_modu)
    
    Y = X_new * A_new
    return X_new, A_new, Y

In [5]:
def alg_extended(net_path, alpha,tsteps,M, epoch, with_truth=False):
    idmap0, idmap_inv0 = [],{}
    for t in tqdm(range(tsteps)):
        print("time:", t)
        idmap, idmap_inv, adj_mat = read_graph(net_path+"%d.gexf" % t)
        
        N = len(idmap)  # number of nodes
        W = Sim(adj_mat, weighted=False)
        if t == 0:
            X, A = np.random.rand(N, M), np.diag(np.random.rand(M))
            X, A = np.matrix(X / np.sum(X, axis=0).reshape(1, M)), np.matrix(A / np.sum(A)) 
            Y = X * A

        else:                    # adjustment for changing of nodes 
            reserved_rows = [idmap_inv0[x] for x in idmap0 if x in idmap]   # idmap_inv0 is the mapping of previous time step 
            reserved_place = [idmap_inv[x] for x in idmap0 if x in idmap]
            Y_revise = np.zeros((N, M))   # N could be different from the previous time step
            Y_revise[reserved_place, :] = Y[reserved_rows, :]   
            Y_revise /= np.sum(Y_revise)
            Y = np.matrix(Y_revise)   # reserve rows for nodes at current time step, the rows corresponding to new nodes are filled with 0
            
            num_new, num_old = len(set(idmap) - set(idmap0)),len(reserved_rows) 
            
            # not mentioned on the paper, but are necessary for node changing
            X_revise = np.ones((N, M)) * (1/M)
            X_revise[reserved_place, :] = X[reserved_rows,:]
            X = X_revise
            X = np.matrix(X / np.sum(X, axis=0).reshape(1, M))
            

        X_new, A_new, Y = param_update(X, A, Y, W, t, alpha, epoch=epoch)
        D = np.zeros((N,))
        for i in range(N):
            D[i] = np.sum(Y[i, :])
        D = np.matrix(np.diag(D))
        soft_comm = D.I * X_new * A_new

        comm_pred = np.array(np.argmax(soft_comm, axis=1)).ravel()
        if with_truth:
            comm = np.array([comm_map[idmap[i]] for i in range(N)])
            print("mutual_info:", mutual_info_score(comm, comm_pred))
        s_modu = soft_modularity(soft_comm, W)
        print("soft_modularity: %f" % s_modu)
        

        community_net = A_new * X_new.T * soft_comm
        evolution_net = X.T * soft_comm
        np.save('community_net'+ "%d.npy" %t, community_net)
        np.save('evolution_net' + "%d.npy"%t, evolution_net)
        np.save('community_label' + "%d.npy"%t, comm_pred)
        X, A = X_new, A_new
        idmap0, idmap_inv0 = idmap, idmap_inv

In [ ]:
# set up params
alpha = 0.5
tsteps = 33
epsilon = 1e40
alg_extended("dynamic_data/graph", alpha, tsteps, 80, epoch=30)

### FacetNet 2

The number of nodes and the number of communities both change.

In [ ]:
# find the optimal k by maximaizing the modularity of dynamic graph in different timestamps
# using loucain package to make a approxiamte vlaue of Q* (which is an approximate value of Q) but is of harf partition
opt_k = []
for t in range(tsteps):
    graph = read_gexf('dynamic_data/graph'+str(t)+'.gexf', node_type=type(1))
    louvain = community_louvain.best_partition(graph)
    opt_k.append(max(list(louvain.values())) + 1)

In [ ]:
# FacetNet of changing # of community (k) between iterations
def param_update_change(X, A, Z, W, t, alpha, epoch):
    X_pre, A_pre, Y_pre = X, A, Z 
    N = Z.shape[0]
    M = A_pre.shape[0]

    if t == 0:
        for e in tqdm(range(epoch)):
            W_apprx = np.true_divide(W , (X_pre * A_pre * X_pre.T+epsilon))
            X_new, A_new = X_pre, A_pre
            
            X_new = np.multiply(X_new, np.multiply(W_apprx * X_new, np.sum(A_new, axis=0).reshape(1, M)))
            A_new = np.multiply(A_new, np.diag(np.diag(X_new.T * W_apprx * X_new)))
                
            X_new, A_new = np.matrix(X_new / np.sum(X_new, axis=0).reshape(1, M)), np.matrix(A / np.sum(A_new))
            X_pre, A_pre = X_new, A_new
    else:
        for e in tqdm(range(epoch)):
            W_apprx = np.true_divide(alpha*W+(1-alpha)*Z , (X_pre * A_pre * X_pre.T+epsilon))
            X_new, A_new = X_pre, A_pre

            X_new = np.multiply(X_new, np.multiply(W_apprx * X_new, np.sum(A_new, axis=0).reshape(1, M)))
            A_new = np.multiply(A_new, np.diag(np.diag(X_new.T * W_apprx* X_new)))
            
            X_new, A_new = np.matrix(X_new / np.sum(X_new, axis=0).reshape(1, M)), np.matrix(A / np.sum(A_new))
            X_pre, A_pre = X_new, A_new
    
    Z = X_new * A_new * X_new.T
    return X_new, A_new, Z

In [ ]:
# main proocess of changing # of community between iterations
def alg_extended_change(net_path, alpha, tsteps, M_opt, epoch, with_truth=False):
    idmap0, idmap_inv0 = [],{}
    for t in tqdm(range(tsteps)):
        print("time:", t)
        idmap, idmap_inv, adj_mat = read_graph(net_path+"%d.gexf" % t)

        N = len(idmap)  # number of nodes
        #W = Sim(adj_mat, weighted=False)
        W = Sim(adj_mat, weighted=True)
        M = M_opt[t]
        
        if t == 0:
            X, A = np.random.rand(N, M), np.diag(np.random.rand(M))
            X, A = np.matrix(X / np.sum(X, axis=0).reshape(1, M)), np.matrix(A / np.sum(A)) # X， A是矩阵
            Z = X * A * X.T

        else:   # adjustment for changing of nodes 
            reserved_rows = [idmap_inv0[x] for x in idmap0 if x in idmap]   
            reserved_place = [idmap_inv[x] for x in idmap0 if x in idmap]
            
            Z_revise = np.zeros((N, N))           
            for i in range(len(reserved_place)):
                for j in range(len(reserved_place)):
                    Z_revise[reserved_place[i], reserved_place[i]] = Z[reserved_rows[i], reserved_rows[j]] 
            Z_revise /= np.sum(Z_revise)
            Z = np.matrix(Z_revise)
            
            num_new, num_old = len(set(idmap) - set(idmap0)), len(reserved_rows) 
            # X with deleting/adding nodes but reserve the last M, required for computing evolutionnet
            last_M = X.shape[1]
            X_pre = np.ones((N, last_M)) * (1/last_M)
            X_pre[reserved_place, :] = X[reserved_rows,:]
            X_pre = np.matrix(X_pre / (np.sum(X_pre, axis=0).reshape(1, last_M)+ 1e-40))
            
            
            # not mentioned on the paper, but are necessary for node changing
            X_revise = np.ones((N, M)) * (1/M)
            M_min = min(M_opt[t-1], M_opt[t])
            X_revise[reserved_place, :M_min] = X[reserved_rows, :M_min]
            X = X_revise
            X = np.matrix(X / np.sum(X, axis=0).reshape(1, M))

            A_revise = np.diag([1/M for i in range(M)])
            A_revise[:M_min, :M_min] = A[:M_min, :M_min]
            A = A_revise
            A = np.matrix(A / np.sum(A_new))
            # print(t, X.shape, A.shape)
            #X *= num_old/(num_old+num_new)  # 这不是不满足每行的和为1了嘛？
            #X = np.pad(X, ((0, num_new), (0, 0)), mode='constant', constant_values=(1/num_new, 1/num_new))
        
        X_new, A_new, Z = param_update_change(X, A, Z, W, t, alpha, epoch=epoch)
            
        Y = X_new * A_new
        D = np.zeros((N,))
        for i in range(N):
            D[i] = np.sum(Y[i, :])+epsilon
        D = np.matrix(np.diag(D))
        soft_comm = D.I * X_new * A_new

        comm_pred = np.array(np.argmax(soft_comm, axis=1)).ravel()
        if with_truth:
            comm = np.array([comm_map[idmap[i]] for i in range(N)])
            print("mutual_info:", mutual_info_score(comm, comm_pred))
        s_modu = soft_modularity(soft_comm, W)
        print("soft_modularity: %f" % s_modu)
        
        community_net = A_new * X_new.T * soft_comm
        if t>0:
            evolution_net = X_pre.T * soft_comm
            np.save('facet_result/weighted_evolution_net' + "%d.npy"%t, evolution_net)
        np.save('facet_result/weighted_community_net'+ "%d.npy" %t, community_net)
        np.save('facet_result/weighted_community_label' + "%d.npy"%t, comm_pred)
        
        X, A = X_new, A_new
        idmap0, idmap_inv0 = idmap, idmap_inv

In [ ]:
alpha = 0.5
tsteps = 33
epsilon=1e-40
alg_extended_change("dynamic_data/graph", alpha, tsteps, opt_k, epoch=60)